In [1]:
# packages
import sympy
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Operators

In [2]:
# initial quadratures, non cummuting
q0, p0 = sympy.symbols('q_0 p_0', commutative=False)

# circuit parameters
r, theta, c, s = sympy.symbols('r_0 theta_0 c_0 s_0')

# q component of the gates: squeezing, rotation, position displacement and cubic phase gate
Sq = q0 * sympy.exp(r)
Rq = q0 * sympy.cos(theta) + p0 * sympy.sin(theta)
Xq = q0 + c
Cq = q0

# p component of the gates: squeezing, roatation, position displacement and cubic phase gate
Sp = p0 * sympy.exp(-r)
Rp = -q0 * sympy.sin(theta) + p0 * sympy.cos(theta)
Xp = p0
Cp = q0**2 * (-3*s) + p0*s

### Squeezing gate

$S(r) = \exp\left( i \frac{r}{2} \left(\hat{q} \hat{p} + \hat{p} \hat{q} \right)\right)$

In [3]:
Sq

q_0*exp(r_0)

In [4]:
Sp

p_0*exp(-r_0)

### Rotation gate

$R(\theta) = \exp\left( i \frac{\theta}{2} \left( \hat{q}^2 + \hat{p}^2 \right) \right)$ 

In [5]:
Rq

p_0*sin(theta_0) + q_0*cos(theta_0)

In [6]:
Rp

p_0*cos(theta_0) - q_0*sin(theta_0)

### Position displacement gate

$X(c) = \exp\left(-ic \hat{p}\right)$

In [7]:
Xq

c_0 + q_0

In [8]:
Xp

p_0

### Cubic phase gate 

$C(s) = \exp\left( -is\hat{q}^3 \right)$

In [9]:
Cq

q_0

In [10]:
Cp

p_0*s_0 - 3*q_0**2*s_0

## Circuit preparation
$
\begin{equation}
    \begin{pmatrix}
        \hat{q}_0 \\
        \hat{p}_0
    \end{pmatrix} \mapsto 
    \begin{pmatrix}
        \hat{q}_1 \\
        \hat{p}_1
    \end{pmatrix} = 
    \prod_{i=1}^{n_{\text{Blocks}}}  S(r_i) C(s_i) R(\theta_i) X(c_i) 
    \begin{pmatrix}
        \hat{q} \\
        \hat{p} 
    \end{pmatrix}
    X(c_i)^\dagger R(\theta_i)^\dagger C(s_i)^\dagger S(r_i)^\dagger 
\end{equation}
$

### One round cricuit

In [11]:
# q and p component for one round of application of the optimisation circuit
q1 = Xq.subs({q0: Rq, p0: Rp}, simultaneous=True).subs({q0: Cq, p0: Cp}, simultaneous=True).subs({q0: Sq, p0: Sp}, simultaneous=True)
p1 = Xp.subs({p0: Rp}, simultaneous=True).subs({q0: Cq, p0: Cp}, simultaneous=True).subs({q0: Sq, p0: Sp}, simultaneous=True)

In [12]:
q1

c_0 + q_0*exp(r_0)*cos(theta_0) + (p_0*s_0*exp(-r_0) - 3*q_0**2*s_0*exp(2*r_0))*sin(theta_0)

In [13]:
p1

-q_0*exp(r_0)*sin(theta_0) + (p_0*s_0*exp(-r_0) - 3*q_0**2*s_0*exp(2*r_0))*cos(theta_0)

### Arbirtrary rounds

In [14]:
num_blocks = 2

r, theta, c, s = sympy.symbols('r_0 theta_0 c_0 s_0')

Sq = q0 * sympy.exp(r)
Rq = q0 * sympy.cos(theta) + p0 * sympy.sin(theta)
Xq = q0 + c
Cq = q0

Sp = p0 * sympy.exp(-r)
Rp = -q0 * sympy.sin(theta) + p0 * sympy.cos(theta)
Xp = p0
Cp = q0**2 * (-3*s) + p0*s

q1 = Xq.subs({q0: Rq, p0: Rp}, simultaneous=True).subs({q0: Cq, p0: Cp}, simultaneous=True).subs({q0: Sq, p0: Sp}, simultaneous=True)
p1 = Xp.subs({p0: Rp}, simultaneous=True).subs({q0: Cq, p0: Cp}, simultaneous=True).subs({q0: Sq, p0: Sp}, simultaneous=True)

for i in range(num_blocks-1): 
    var_string = f'r_{i+1} theta_{i+1} c_{i+1} s_{i+1}'

    r, theta, c, s = sympy.symbols(var_string)

    Sq = q0 * sympy.exp(r)
    Rq = q0 * sympy.cos(theta) + p0 * sympy.sin(theta)
    Xq = q0 + c
    Cq = q0
    
    Sp = p0 * sympy.exp(-r)
    Rp = -q0 * sympy.sin(theta) + p0 * sympy.cos(theta)
    Xp = p0
    Cp = q0**2 * (-3*s) + p0*s
    
    qc = Xq.subs({q0: Rq, p0: Rp}, simultaneous=True).subs({q0: Cq, p0: Cp}, simultaneous=True).subs({q0: Sq, p0: Sp}, simultaneous=True)
    pc = Xp.subs({p0: Rp}, simultaneous=True).subs({q0: Cq, p0: Cp}, simultaneous=True).subs({q0: Sq, p0: Sp}, simultaneous=True)

    q1 = q1.subs({q0: qc, p0: pc}, simultaneous=True)
    p1 = p1.subs({q0: qc, p0: pc}, simultaneous=True) 

In [15]:
q1

c_0 + (s_0*(-q_0*exp(r_1)*sin(theta_1) + (p_0*s_1*exp(-r_1) - 3*q_0**2*s_1*exp(2*r_1))*cos(theta_1))*exp(-r_0) - 3*s_0*(c_1 + q_0*exp(r_1)*cos(theta_1) + (p_0*s_1*exp(-r_1) - 3*q_0**2*s_1*exp(2*r_1))*sin(theta_1))**2*exp(2*r_0))*sin(theta_0) + (c_1 + q_0*exp(r_1)*cos(theta_1) + (p_0*s_1*exp(-r_1) - 3*q_0**2*s_1*exp(2*r_1))*sin(theta_1))*exp(r_0)*cos(theta_0)

In [16]:
p1

(s_0*(-q_0*exp(r_1)*sin(theta_1) + (p_0*s_1*exp(-r_1) - 3*q_0**2*s_1*exp(2*r_1))*cos(theta_1))*exp(-r_0) - 3*s_0*(c_1 + q_0*exp(r_1)*cos(theta_1) + (p_0*s_1*exp(-r_1) - 3*q_0**2*s_1*exp(2*r_1))*sin(theta_1))**2*exp(2*r_0))*cos(theta_0) - (c_1 + q_0*exp(r_1)*cos(theta_1) + (p_0*s_1*exp(-r_1) - 3*q_0**2*s_1*exp(2*r_1))*sin(theta_1))*exp(r_0)*sin(theta_0)

## Calculating with optimal parameters

In [20]:
sympy.expand(q1)

c_0 - 3*c_1**2*s_0*exp(2*r_0)*sin(theta_0) - 6*c_1*p_0*s_0*s_1*exp(2*r_0)*exp(-r_1)*sin(theta_0)*sin(theta_1) + 18*c_1*q_0**2*s_0*s_1*exp(2*r_0)*exp(2*r_1)*sin(theta_0)*sin(theta_1) - 6*c_1*q_0*s_0*exp(2*r_0)*exp(r_1)*sin(theta_0)*cos(theta_1) + c_1*exp(r_0)*cos(theta_0) - 3*p_0**2*s_0*s_1**2*exp(2*r_0)*exp(-2*r_1)*sin(theta_0)*sin(theta_1)**2 + 18*p_0*q_0**2*s_0*s_1**2*exp(2*r_0)*exp(r_1)*sin(theta_0)*sin(theta_1)**2 - 6*p_0*q_0*s_0*s_1*exp(2*r_0)*sin(theta_0)*sin(theta_1)*cos(theta_1) + p_0*s_0*s_1*exp(-r_0)*exp(-r_1)*sin(theta_0)*cos(theta_1) + p_0*s_1*exp(r_0)*exp(-r_1)*sin(theta_1)*cos(theta_0) - 27*q_0**4*s_0*s_1**2*exp(2*r_0)*exp(4*r_1)*sin(theta_0)*sin(theta_1)**2 + 18*q_0**3*s_0*s_1*exp(2*r_0)*exp(3*r_1)*sin(theta_0)*sin(theta_1)*cos(theta_1) - 3*q_0**2*s_0*s_1*exp(-r_0)*exp(2*r_1)*sin(theta_0)*cos(theta_1) - 3*q_0**2*s_0*exp(2*r_0)*exp(2*r_1)*sin(theta_0)*cos(theta_1)**2 - 3*q_0**2*s_1*exp(r_0)*exp(2*r_1)*sin(theta_1)*cos(theta_0) - q_0*s_0*exp(-r_0)*exp(r_1)*sin(theta_0)*sin

In [24]:
sympy.collect(sympy.collect(sympy.expand(q1), q0), p0)


c_0 - 3*c_1**2*s_0*exp(2*r_0)*sin(theta_0) + c_1*exp(r_0)*cos(theta_0) - 3*p_0**2*s_0*s_1**2*exp(2*r_0)*exp(-2*r_1)*sin(theta_0)*sin(theta_1)**2 + p_0*(-6*c_1*s_0*s_1*exp(2*r_0)*exp(-r_1)*sin(theta_0)*sin(theta_1) + s_0*s_1*exp(-r_0)*exp(-r_1)*sin(theta_0)*cos(theta_1) + s_1*exp(r_0)*exp(-r_1)*sin(theta_1)*cos(theta_0)) - 27*q_0**4*s_0*s_1**2*exp(2*r_0)*exp(4*r_1)*sin(theta_0)*sin(theta_1)**2 + 18*q_0**3*s_0*s_1*exp(2*r_0)*exp(3*r_1)*sin(theta_0)*sin(theta_1)*cos(theta_1) + q_0**2*(18*c_1*s_0*s_1*exp(2*r_0)*exp(2*r_1)*sin(theta_0)*sin(theta_1) + 18*p_0*s_0*s_1**2*exp(2*r_0)*exp(r_1)*sin(theta_0)*sin(theta_1)**2 - 3*s_0*s_1*exp(-r_0)*exp(2*r_1)*sin(theta_0)*cos(theta_1) - 3*s_0*exp(2*r_0)*exp(2*r_1)*sin(theta_0)*cos(theta_1)**2 - 3*s_1*exp(r_0)*exp(2*r_1)*sin(theta_1)*cos(theta_0)) + q_0*(-6*c_1*s_0*exp(2*r_0)*exp(r_1)*sin(theta_0)*cos(theta_1) - 6*p_0*s_0*s_1*exp(2*r_0)*sin(theta_0)*sin(theta_1)*cos(theta_1) - s_0*exp(-r_0)*exp(r_1)*sin(theta_0)*sin(theta_1) + exp(r_0)*exp(r_1)*cos(the